In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
raw_train_data = pd.read_csv("../input/titanic/train.csv")
raw_test_data = pd.read_csv("../input/titanic/test.csv")

train_data = raw_train_data.copy()
test_data = raw_test_data.copy()
combined = [train_data, test_data]


In [ ]:
train_data.describe(include = 'all')

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data = train_data.drop(['Ticket', 'Cabin'], axis=1)
test_data = test_data.drop(['Ticket', 'Cabin'], axis=1)
combined = [train_data, test_data]

In [ ]:
train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)


In [ ]:
train_data[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)


In [ ]:
train_data[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)


In [ ]:
train_data[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)


In [ ]:
for df in combined:
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_data['Title'], train_data['Sex'])

In [ ]:
for df in combined:
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
train_data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
for df in combined:
    df['Title'] = df['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    df['Title'] = df['Title'].fillna(0)

train_data.head()

In [ ]:
train_data = train_data.drop(['Name'], axis=1)
test_data = test_data.drop(['Name'], axis=1)
combined = [train_data, test_data]
train_data.head()

In [ ]:
for df in combined:
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_data.head()

In [ ]:
med_train = train_data['Age'].median()
med_test= test_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(value = med_train)
test_data['Age'] = test_data['Age'].fillna(value = med_test)
combined = [train_data, test_data]

train_data

In [ ]:
for df in combined:
    df['IsAlone'] = 0
    df.loc[df['SibSp'] + df['Parch'] ==0, 'IsAlone'] = 1

train_data[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
train_data = train_data.drop(['Parch', 'SibSp'], axis=1)
test_data = test_data.drop(['Parch', 'SibSp'], axis=1)
combined = [train_data, test_data]

train_data.head()

In [ ]:
most_freq_embarked = train_data.Embarked.dropna().mode()[0]
most_freq_embarked

In [ ]:
for df in combined:
    df['Embarked'] = df['Embarked'].fillna(most_freq_embarked)
    df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
train_data[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
test_data['Fare'].fillna(test_data['Fare'].dropna().median(), inplace=True)

In [ ]:
fare_quartiles = pd.qcut(train_data['Fare'], 4, retbins = True)[1]
q1 = quartiles[1]
q2 = quartiles[2]
q3 = quartiles[3]
q4 = quartiles[4]
for df in combined:
    df.loc[ df['Fare'] <= q1, 'Fare'] = 0
    df.loc[(df['Fare'] > q1) & (df['Fare'] <= q2), 'Fare'] = 1
    df.loc[(df['Fare'] > q2) & (df['Fare'] <= q3), 'Fare']   = 2
    df.loc[ df['Fare'] > q3, 'Fare'] = 3
    df['Fare'] = df['Fare'].astype(int)


combined = [train_data, test_data]
train_data.head()

In [ ]:
age_quartiles = pd.qcut(train_data['Age'], 4, retbins = True)[1]
a_q1 = age_quartiles[1]
a_q2 = age_quartiles[2]
a_q3 = age_quartiles[3]
a_q4 = age_quartiles[4]

for df in combined:
    df.loc[ df['Age'] <= a_q1, 'Age'] = 0
    df.loc[(df['Age'] > a_q1) & (df['Age'] <= a_q2), 'Age'] = 1
    df.loc[(df['Age'] > a_q2) & (df['Age'] <= a_q3), 'Age'] = 2
    df.loc[ df['Age'] > a_q3, 'Age'] = 3
    df['Age'] = df['Age'].astype(int)


combined = [train_data, test_data]
train_data

In [ ]:
for df in combined:
    df['Age*Pclass'] = df['Age'] * df['Pclass']
train_data[['Age*Pclass', 'Survived']].groupby(['Age*Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

train_data.head()

In [ ]:
for df in combined:
    df.set_index('PassengerId', inplace = True)
test_data

In [ ]:
X_train = train_data.drop("Survived", axis=1)
y_train = train_data["Survived"]
X_test = test_data

In [ ]:
survived_count = int(np.sum(train_data['Survived']))
survived_count

In [ ]:
not_survived_count = 0
indices_to_remove = []
for i in range(train_data['Survived'].shape[0]):
    if train_data['Survived'].iloc[i] == 0:
        not_survived_count += 1
        if not_survived_count > survived_count:
            indices_to_remove.append(i)
            
            
indices_to_remove


In [ ]:
X_train_bal = X_train.drop(index = indices_to_remove, axis = 0)

X_train_bal 

In [ ]:
y_train_bal = y_train.drop(index = indices_to_remove, axis = 0)
y_train_bal


In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_bal, y_train_bal)
y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train_bal, y_train_bal) * 100, 2)
acc_log

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, y_train) * 100, 2)
acc_knn

In [ ]:
y_pred

In [ ]:
result = pd.DataFrame({"PassengerId": raw_test_data['PassengerId'] ,
                      "Survived": y_pred})
result
result.to_csv('sub5.csv',index=False)